In [136]:
import pytorch_warmup as warmup
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [150]:
model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# factor = 0.6warmup_
# scheduler1 = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: (1 + epoch) * factor ** (warmup_epochs - epoch))
scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, end_factor=1.0, total_iters=5)
scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[warmup_epochs - 1])
lrs = []

for epoch in range(30):
    lrs.append(optimizer.param_groups[0]['lr'])
    optimizer.step()
    scheduler.step()

In [153]:
model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

factor = 0.4
scheduler1 = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: (1 + epoch) * factor ** (warmup_epochs - epoch))
# scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, end_factor=1.0, total_iters=5)
scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[warmup_epochs - 1])
lrs2 = []

for epoch in range(30):
    lrs2.append(optimizer.param_groups[0]['lr'])
    optimizer.step()
    scheduler.step()

In [162]:
model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

start = 1e-5
end = 1e-3
steps = 5
factor = (end / start) ** (1 / (steps - 1))

scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=factor)
scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[warmup_epochs])
lrs3 = []

for epoch in range(30):
    lrs3.append(optimizer.param_groups[0]['lr'])
    optimizer.step()
    scheduler.step()

In [156]:
lrs[:5]

In [157]:
lrs2[:5]

In [164]:
lrs3[:7]

In [167]:
plt.plot(lrs)
plt.plot(lrs2)
plt.plot(lrs3)

In [132]:
warmup_epochs = 5
base_lr = 1e-3
warmup_factor = 1e-5
lrs = []

model = nn.Linear(1, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=base_lr)
warmup = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=warmup_factor, end_factor=1, total_iters=warmup_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [133]:
for epoch in range(30):
    optimizer.step()

    if epoch < warmup_epochs:
        warmup.step()
    else:
        scheduler.step()
        
    lrs.append(optimizer.param_groups[0]['lr'])

In [134]:
lrs

In [135]:
plt.plot(lrs)

In [25]:
model = torch.nn.Linear(10, 32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [28]:
warmup_period = 5
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
warmup_scheduler = warmup.LinearWarmup(optimizer, warmup_period)
lrs1, lrs2, lrs3 = [], [], []

for epoch in range(30):
    with warmup_scheduler.dampening():
        if warmup_scheduler.last_step + 1 >= warmup_period:
            lr_scheduler.step()
            
        lrs1.append(optimizer.param_groups[0]['lr'])
        lrs2.append(lr_scheduler.get_last_lr()[0])
        lrs3.append(warmup_scheduler.last_step)

In [29]:
plt.plot(lrs1)

In [30]:
plt.plot(lrs2)

In [31]:
plt.plot(lrs3)